In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import io 
import zipfile
import xml.etree.ElementTree as et 
import json
import requests

In [2]:
api_num = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/apinumber.txt', sep='\s+', header=None,
                                names=['column_name'])
api_number = api_num['column_name']

In [3]:
def get_corpcode(crtfc_key): 
    params = {'crtfc_key':crtfc_key}
    url = "https://opendart.fss.or.kr/api/corpCode.xml" 
    items_en = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    
    ## Column명을 한글로 하려면 아래부분 쌍따옴표안을 수정하면 됨 
    items_kr = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    results = requests.get(url,params=params) 
    uzfile = zipfile.ZipFile(io.BytesIO(results.content))
    final = uzfile.open(uzfile.namelist()[0]) 
    root = et.fromstring(final.read().decode('utf-8')) 
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: 
            data.append([]) 
            for item in items_en: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns = items_kr) 
    ##print(df) 
    return df

In [4]:
# dart에 있는 모든 기업 리스트 추출
crtfc_key = api_number[0]
code_df = get_corpcode(crtfc_key)
code_df.head()

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630


In [5]:
# 다트에 재무제표 올라와 있는 기업 개수 : 3418
code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3434 entries, 0 to 3433
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   corp_code    3434 non-null   object
 1   corp_name    3434 non-null   object
 2   stock_code   3434 non-null   object
 3   modify_date  3434 non-null   object
dtypes: object(4)
memory usage: 107.4+ KB


In [6]:
# 반도체기업 이름 추출
company_name_df = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/반도체 기업 목록.txt', sep='\s+', header=None,
                                names=['column_index', 'column_name'])

In [7]:
# 전체 재무제표 데이터 중 반도체 기업 이름 확인
company_names = company_name_df['column_index'].values.tolist()
print(company_names[:10])

['큐에스아이', '예스티', '미코', '신성이엔지', '코세스', '디엔에프', '로체시스템즈', '휘닉스소재', '주성엔지니어링', '라온테크']


In [8]:
# 반도체기업 기업코드 추출
df_ban = code_df[code_df['corp_name'].isin(company_names)]
df_ban.head(10)

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,104110,20171208
761,00625942,아이원스,114810,20220117
788,00484682,엘오티베큠,083310,20211123
789,00492894,젬백스,082270,20211124
805,00158219,시그네틱스,033170,20211229
810,00372226,티에스이,131290,20211201
840,00325112,프로텍,053610,20211209
912,01596425,SK스퀘어,402340,20220105
930,00530413,코디엠,224060,20211210
969,00530556,예스티,122640,20211207


## 년도 바꿀 때 여기부터 다시 실행 후 년도, 보고서 넘버 변경

In [175]:
# 가나다 순으로 정렬
df_bando = df_ban.sort_values('corp_name', ascending=True)
df_bando['corp_code'] = df_bando['corp_code'].apply(lambda x : x.strip())
df_bando['corp_name'] = df_bando['corp_name'].apply(lambda x : x.strip())
df_bando['stock_code'] = df_bando['stock_code'].apply(lambda x : x.strip())
df_bando.head(10)

,corp_code,corp_name,stock_code,modify_date
2106,00160843,DB하이텍,000990,20220216
3197,00540863,GST,083450,20220217
2225,00572905,ISC,095340,20211203
3038,00589127,KEC,092220,20220228
2472,00276834,KMH하이텍,052900,20220221
1045,00301246,SFA반도체,036540,20211207
970,01596425,SK스퀘어,402340,20220105
3247,00164779,SK하이닉스,000660,20220308
1627,00440712,WI,073570,20211208
976,00377018,기가레인,049080,20211201


In [176]:
# 127개 기업
df_bando.nunique()

corp_code      127
corp_name      126
stock_code     127
modify_date     46
dtype: int64

In [177]:
# 반도체 기업 리스트로 추출
company_code = df_bando['corp_code'].values.tolist()
print(company_code[:10])

['00160843', '00540863', '00572905', '00589127', '00276834', '00301246', '01596425', '00164779', '00440712', '00377018']


In [178]:
path = "/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/재무제표파일/"

In [179]:
def read_json(code):
  base_url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?"
  params = {'crtfc_key': api_number, 'corp_code': code , 'bsns_year': '2021', 'reprt_code': '11011'}
  res = requests.get(base_url, params=params)
  print(res.text)
  jsonObject = json.loads(res.text)
  status = jsonObject.get("status")
  if status == "013":
    return False
  elif status == "000":
    return True

df_bando['status'] = df_bando['corp_code'].apply(lambda x: read_json(x))

df_bando

{"status":"000","message":"정상","list":[{"rcept_no":"20220321001280","reprt_code":"11011","bsns_year":"2021","corp_code":"00160843","stock_code":"000990","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"유동자산","thstrm_nm":"제 69 기","thstrm_dt":"2021.12.31 현재","thstrm_amount":"815,161,899,425","frmtrm_nm":"제 68 기","frmtrm_dt":"2020.12.31 현재","frmtrm_amount":"459,208,127,449","bfefrmtrm_nm":"제 67 기","bfefrmtrm_dt":"2019.12.31 현재","bfefrmtrm_amount":"426,263,421,167","ord":"1"},{"rcept_no":"20220321001280","reprt_code":"11011","bsns_year":"2021","corp_code":"00160843","stock_code":"000990","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"비유동자산","thstrm_nm":"제 69 기","thstrm_dt":"2021.12.31 현재","thstrm_amount":"734,775,996,429","frmtrm_nm":"제 68 기","frmtrm_dt":"2020.12.31 현재","frmtrm_amount":"719,373,069,732","bfefrmtrm_nm":"제 67 기","bfefrmtrm_dt":"2019.12.31 현재","bfefrmtrm_amount":"662,626,388,404","ord":"3"},{"rcept_no":"20220321001280","

,corp_code,corp_name,stock_code,modify_date,status
2106,00160843,DB하이텍,000990,20220216,True
3197,00540863,GST,083450,20220217,True
2225,00572905,ISC,095340,20211203,True
3038,00589127,KEC,092220,20220228,True
2472,00276834,KMH하이텍,052900,20220221,True
...,...,...,...,...,...
2548,00159254,한국전자홀딩스,006200,20220225,False
1886,00161383,한미반도체,042700,20211208,True
2865,00488989,한양디지텍,078350,20220228,True
2143,01032413,해성디에스,195870,20220216,True


In [180]:
print(df_bando.shape)

(127, 5)


In [181]:
df_bando

,corp_code,corp_name,stock_code,modify_date,status
2106,00160843,DB하이텍,000990,20220216,True
3197,00540863,GST,083450,20220217,True
2225,00572905,ISC,095340,20211203,True
3038,00589127,KEC,092220,20220228,True
2472,00276834,KMH하이텍,052900,20220221,True
...,...,...,...,...,...
2548,00159254,한국전자홀딩스,006200,20220225,False
1886,00161383,한미반도체,042700,20211208,True
2865,00488989,한양디지텍,078350,20220228,True
2143,01032413,해성디에스,195870,20220216,True


In [182]:
# df_copy = df_bando
# df_copy

In [183]:
# df_False = df_copy[(df_copy['status'] == False)]
# # status_mask = df_copy[df_copy['status'] == 'False'].index
# # print(status_mask)
# df_False

In [184]:
# df_false.to_excel('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/재무제표파일/2017_1분기_False.xlsx')

In [185]:
df_bando = df_bando.reset_index(drop=True)
df_bando = df_bando.drop(['stock_code', 'status','modify_date'], axis=1)
# df_bando.to_csv(path + "corpcode_in_dart.csv")
df_bando

,corp_code,corp_name
0,00160843,DB하이텍
1,00540863,GST
2,00572905,ISC
3,00589127,KEC
4,00276834,KMH하이텍
...,...,...
122,00159254,한국전자홀딩스
123,00161383,한미반도체
124,00488989,한양디지텍
125,01032413,해성디에스


In [186]:
def read_json(code):
  base_url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?"
  params = {'crtfc_key': api_number, 'corp_code': code , 'bsns_year': '2021', 'reprt_code': '11011'}
  res = requests.get(base_url, params=params)
  jsonObject = json.loads(res.text).get("list")
  df = pd.DataFrame.from_dict(jsonObject)
  return df

all_df = pd.DataFrame()
for c in df_bando['corp_code']:
  df = read_json(c)
  all_df = pd.concat([df, all_df], axis=0)

all_df = all_df.reset_index(drop=False)
all_df = all_df.rename({'index':'no'},axis='columns')
all_df.to_csv(path + "반도체기업_2021_4분기사업보고서.csv")
all_df.head()

,no,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,...,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,0,20220316001232,11011,2021,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 22 기,2021.12.31 현재,"22,242,509,767",제 21 기,2020.12.31 현재,"25,247,195,172",제 20 기,2019.12.31 현재,"21,589,666,468",2
1,1,20220316001232,11011,2021,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 22 기,2021.12.31 현재,"5,685,651,742",제 21 기,2020.12.31 현재,"5,111,472,624",제 20 기,2019.12.31 현재,"8,436,023,966",4
2,2,20220316001232,11011,2021,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 22 기,2021.12.31 현재,"42,173,161,509",제 21 기,2020.12.31 현재,"44,603,667,796",제 20 기,2019.12.31 현재,"44,270,690,434",6
3,3,20220316001232,11011,2021,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 22 기,2021.12.31 현재,"15,355,409,911",제 21 기,2020.12.31 현재,"15,124,690,432",제 20 기,2019.12.31 현재,"15,594,967,177",8
4,4,20220316001232,11011,2021,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 22 기,2021.12.31 현재,"1,743,344,458",제 21 기,2020.12.31 현재,"1,433,893,988",제 20 기,2019.12.31 현재,"1,386,638,048",10


-- 여기까지

In [138]:
all_df.head()

,no,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,...,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,0,20180619000096,11011,2017,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 18 기,2017.12.31 현재,"9,199,982,624",제 17 기,2016.12.31 현재,"8,061,959,358",제 16 기,2015.12.31 현재,"5,062,428,587",2
1,1,20180619000096,11011,2017,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 18 기,2017.12.31 현재,"34,939,480,087",제 17 기,2016.12.31 현재,"37,421,081,602",제 16 기,2015.12.31 현재,"48,560,273,560",4
2,2,20180619000096,11011,2017,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 18 기,2017.12.31 현재,"44,139,462,711",제 17 기,2016.12.31 현재,"51,934,968,319",제 16 기,2015.12.31 현재,"53,622,702,147",6
3,3,20180619000096,11011,2017,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 18 기,2017.12.31 현재,"11,192,976,187",제 17 기,2016.12.31 현재,"21,039,462,376",제 16 기,2015.12.31 현재,"12,472,885,395",8
4,4,20180619000096,11011,2017,00369569,050090,OFS,재무제표,BS,재무상태표,...,제 18 기,2017.12.31 현재,"2,127,767,070",제 17 기,2016.12.31 현재,"1,592,538,322",제 16 기,2015.12.31 현재,"19,237,580,080",10


In [115]:
all_copy2= all_df[['corp_code',	'fs_nm','account_nm', 'thstrm_amount','thstrm_dt']]
all_copy2.head()

,corp_code,fs_nm,account_nm,thstrm_amount,thstrm_dt
0,00369569,연결재무제표,유동자산,"11,141,014,889",2018.12.31 현재
1,00369569,연결재무제표,비유동자산,"8,167,789,003",2018.12.31 현재
2,00369569,연결재무제표,자산총계,"33,553,803,892",2018.12.31 현재
3,00369569,연결재무제표,유동부채,"2,482,815,865",2018.12.31 현재
4,00369569,연결재무제표,비유동부채,"3,196,535,306",2018.12.31 현재


In [116]:
all_copy2.pivot(index='corp_code', columns = 'account_nm', values = 'thstrm_amount')

ValueError: Index contains duplicate entries, cannot reshape

In [117]:
df1 = pd.pivot_table(all_copy2, index='corp_code', columns = 'account_nm', values = 'thstrm_amount', aggfunc=np.sum, fill_value=0)
df1 = df1.reset_index()
df1

account_nm,corp_code,당기순이익,매출액,법인세차감전 순이익,부채총계,비유동부채,비유동자산,영업이익,유동부채,유동자산,이익잉여금,자본금,자본총계,자산총계
0,00118275,"14,684,624,5941,969,093,043","195,994,249,82080,608,154,056","20,596,894,9362,112,407,001","66,411,217,86847,391,604,984","17,243,119,45512,758,747,449","121,828,658,188110,878,286,281","21,524,010,6411,016,705,408","49,168,098,41334,632,857,535","88,660,278,24948,613,207,695","78,281,587,60658,400,990,396","17,248,392,50017,248,392,500","144,077,718,569112,099,888,992","210,488,936,437159,491,493,976"
1,00118804,"47,958,363,78639,569,460,423","827,231,161,595632,154,575,017","61,731,289,30050,444,038,189","454,557,635,269357,702,915,764","98,752,977,02475,064,201,791","396,843,579,488404,311,594,501","70,984,302,81551,219,282,792","355,804,658,245282,638,713,973","363,608,168,087241,521,888,350","141,154,291,908148,663,183,257","25,707,247,00025,707,247,000","305,894,112,306288,130,567,087","760,451,747,575645,833,482,851"
2,00121686,"133,221,552,392-10,347,727,070","706,474,289,010335,998,709,628","199,202,550,6852,101,259,373","933,975,437,765173,258,336,183","53,738,955,2013,450,517,565","198,756,954,436159,635,719,345","173,689,313,9933,468,307,365","328,957,909,205169,807,818,618","209,596,389,043116,873,099,907","291,982,133,17576,264,032,880","10,903,844,50010,903,844,500","848,836,659,574103,250,483,069","1,782,812,097,339276,508,819,252"
3,00126380,"44,344,857,000,00032,815,127,000,000","243,771,415,000,000170,381,870,000,000","61,159,958,000,00044,398,855,000,000","91,604,067,000,00046,033,232,000,000","22,522,557,000,0002,888,179,000,000","164,659,820,000,000138,981,902,000,000","58,886,669,000,00043,699,451,000,000","69,081,510,000,00043,145,053,000,000","174,697,424,000,00080,039,455,000,000","242,698,956,000,000166,555,532,000,000","897,514,000,000897,514,000,000","247,753,177,000,000172,988,125,000,000","339,357,244,000,000219,021,357,000,000"
4,00136271,"10,698,129,762-6,749,699,724","424,654,240,360320,628,826,301","-20,412,237,507-29,638,629,490","299,061,478,628245,554,573,766","27,051,173,31325,288,889,955","253,295,611,612247,060,691,616","-4,152,168,302-10,522,501,556","272,010,305,315220,265,683,811","159,849,487,47997,350,599,542","-81,842,619,840","86,606,856,500","98,856,717,392","413,145,099,091344,411,291,158"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,01113754,"14,077,175,15411,783,730,037","94,181,913,97673,781,548,136","16,957,322,51214,457,670,345","26,492,784,26419,654,369,449",00,"41,928,486,75041,794,732,000","17,282,133,73014,906,274,024","26,492,784,26419,654,369,449","39,388,677,77229,109,190,427","32,964,154,11631,461,896,371","4,562,490,0004,562,490,000","54,824,380,25851,249,552,978","81,317,164,52270,903,922,427"
106,01135941,"86,860,388,55586,409,359,707","649,312,684,893649,002,715,840","109,813,867,132109,304,431,774","143,129,539,013143,216,695,494","19,162,792,08819,162,792,088","180,630,413,482181,151,235,519","105,894,511,995105,357,467,856","123,966,746,925124,053,903,406","331,715,602,680330,986,780,448","173,678,038,574173,218,334,077","20,636,718,00020,636,718,000","369,216,477,149368,921,320,473","512,346,016,162512,138,015,967"
107,01170865,"51,649,368",0,"66,217,140","762,444,259","762,365,759",0,"-58,214,280","78,500","11,200,669,614","16,552,099","551,000,000","10,438,225,355","11,200,669,614"
108,01188378,"4,815,529,6394,771,302,782","22,371,734,51120,630,140,773","5,582,199,1405,567,578,810","2,434,741,2191,700,909,642","127,069,115127,069,115","13,382,079,73413,484,157,700","5,054,087,2434,987,527,048","2,307,672,1041,573,840,527","25,118,403,72424,212,366,298","22,746,398,48522,660,330,452","1,800,000,0001,800,000,000","36,065,742,23935,995,614,356","38,500,483,45837,696,523,998"


In [111]:
all_copy3 = all_copy2.pivot(index='corp_code', columns = 'account_nm', values = 'thstrm_amount')
all_copy3

ValueError: Index contains duplicate entries, cannot reshape

In [103]:
all_df_copy = all_df[['corp_code','account_nm','thstrm_amount']]
all_df_copy.head(20)

,corp_code,account_nm,thstrm_amount
0,00369569,유동자산,"11,141,014,889"
1,00369569,비유동자산,"8,167,789,003"
2,00369569,자산총계,"33,553,803,892"
3,00369569,유동부채,"2,482,815,865"
4,00369569,비유동부채,"3,196,535,306"
5,00369569,부채총계,"5,679,351,171"
6,00369569,자본금,"34,569,471,000"
7,00369569,이익잉여금,"-62,143,537,723"
8,00369569,자본총계,"27,874,452,721"
9,00369569,매출액,"14,927,444,101"


In [104]:
df1 = pd.pivot_table(all_df_copy, index='corp_code', columns = 'account_nm', values = 'thstrm_amount', aggfunc=np.sum, fill_value=0)
df1 = df1.reset_index()
df1.head()

account_nm,corp_code,당기순이익,매출액,법인세차감전 순이익,부채총계,비유동부채,비유동자산,영업이익,유동부채,유동자산,이익잉여금,자본금,자본총계,자산총계
0,00118275,"14,684,624,5941,969,093,043","195,994,249,82080,608,154,056","20,596,894,9362,112,407,001","66,411,217,86847,391,604,984","17,243,119,45512,758,747,449","121,828,658,188110,878,286,281","21,524,010,6411,016,705,408","49,168,098,41334,632,857,535","88,660,278,24948,613,207,695","78,281,587,60658,400,990,396","17,248,392,50017,248,392,500","144,077,718,569112,099,888,992","210,488,936,437159,491,493,976"
1,00118804,"47,958,363,78639,569,460,423","827,231,161,595632,154,575,017","61,731,289,30050,444,038,189","454,557,635,269357,702,915,764","98,752,977,02475,064,201,791","396,843,579,488404,311,594,501","70,984,302,81551,219,282,792","355,804,658,245282,638,713,973","363,608,168,087241,521,888,350","141,154,291,908148,663,183,257","25,707,247,00025,707,247,000","305,894,112,306288,130,567,087","760,451,747,575645,833,482,851"
2,00121686,"133,221,552,392-10,347,727,070","706,474,289,010335,998,709,628","199,202,550,6852,101,259,373","933,975,437,765173,258,336,183","53,738,955,2013,450,517,565","198,756,954,436159,635,719,345","173,689,313,9933,468,307,365","328,957,909,205169,807,818,618","209,596,389,043116,873,099,907","291,982,133,17576,264,032,880","10,903,844,50010,903,844,500","848,836,659,574103,250,483,069","1,782,812,097,339276,508,819,252"
3,00126380,"44,344,857,000,00032,815,127,000,000","243,771,415,000,000170,381,870,000,000","61,159,958,000,00044,398,855,000,000","91,604,067,000,00046,033,232,000,000","22,522,557,000,0002,888,179,000,000","164,659,820,000,000138,981,902,000,000","58,886,669,000,00043,699,451,000,000","69,081,510,000,00043,145,053,000,000","174,697,424,000,00080,039,455,000,000","242,698,956,000,000166,555,532,000,000","897,514,000,000897,514,000,000","247,753,177,000,000172,988,125,000,000","339,357,244,000,000219,021,357,000,000"
4,00136271,"10,698,129,762-6,749,699,724","424,654,240,360320,628,826,301","-20,412,237,507-29,638,629,490","299,061,478,628245,554,573,766","27,051,173,31325,288,889,955","253,295,611,612247,060,691,616","-4,152,168,302-10,522,501,556","272,010,305,315220,265,683,811","159,849,487,47997,350,599,542","-81,842,619,840","86,606,856,500","98,856,717,392","413,145,099,091344,411,291,158"


In [105]:
df1.to_csv(path + "반도체기업_2018_4분기사업보고서.csv")